# Context Manager

`Context Managers` in `Python` allows us to manage the resources (file handlers, sockets, locks etc) without having to explicitly manage them. Say we opened/accessed a resources to perform some operation on it. without context managers, our biggest worry is what will happen, if for some reason, that resource is not closed/released. 

With context managers we don't have to worry about them. It will take care of it for you transparently. Most common method, in this regards is `open` command, which allows us to open a file so that various operations can be performed don it. 

In [8]:
# very bad code and that to without context manager

fp = open("my_text.txt", "w+")
fp.write("Context Managers are great, Context Managers are our Friend.\nSo why not use them")
fp.close()

fp = open("my_text.txt")
print("".join(fp.readlines()))
fp.close()

Context Managers are great, Context Managers are our Friend.
So why not use them


Ok, so why I said its a bad piece of code. First it looks ugly to my eye, as no error handling at all. Lets add the error handling to it. 

In [10]:
try:
    fp = open("my_text.txt", "w+")
    fp.write("Context Managers are great, Context Managers are our Friend.\nSo why not use them")
except Exception as e:
    print(e)
finally:
    fp.close()

try:
    fp = open("my_text.txt")
    print("".join(fp.readlines()))
except Exception as e:
    print(e)
finally:
    fp.close()

Context Managers are great, Context Managers are our Friend.
So why not use them


so, this should have been better,but now, if possible it looks more ugly, and with nearly duble the code, out of which most of the code is being repeated and if we are doing multiple read/write, than its even bad as most of the code could have been is error hanlding which is being repeated multiple times.

So, how can we rectify this situation?

Ofcource, by using context managers we can resolve the issue.

**Syntax:**

```python
with <open the resource> as <res_handler_name>:
    <use the resource_handler>
```

**Example:**

In [13]:
with open("my_text.txt", "w+") as fp:
    fp.write("Context Managers are great, Context Managers are our Friend.\nSo why not use them")

with open("my_text.txt") as fp:
    print("".join(fp.readlines()))

Context Managers are great, Context Managers are our Friend.
So why not use them


`fp` file handler is valid only inside the `with` context block and has no value outside it. We can find this by following code

In [26]:
try:
    with open("my_text.txt", "w+") as fp:
        fp.write(""""Context Managers are great,
Context Managers are our Friend.
So why not use them""")

    with open("my_text.txt") as file_handler:
        print("".join(file_handler.readlines()))
    print(file_handler.name)
    print(file_handler.readlines())
except Exception as e:
    print("* error:", e)

"Context Managers are great,
Context Managers are our Friend.
So why not use them
my_text.txt
* error: I/O operation on closed file.


As we can see from the above example that `context` is not scope, and `scope` is not `context`. We are still able to access few attribues of `fp` such as `name` even after the context block, but not able to access the resource as they are closed outside the context block.

## Gotcha's

- Creating generator within `with` and accessing them later

In [52]:
filename = "my_text.txt"
with open(filename) as f:
    lines = (line.rstrip('\n') for line in f)

print(type(lines))
for l in lines:
    print(l)

<class 'generator'>


ValueError: I/O operation on closed file.

So, what went wrong, we created a generator `lines` and tried to access it after the context. As the resource was already closed at the time of accessing it, thus resulted in error. 

We can do one of the following to fix the situation

In [31]:
file_name = "my_text.txt"

# Fix 1: Using list instead of generator
with open(file_name) as f:
    lines = [line.strip('\n') for line in f]

for l in lines:
    print(l)

print("~^" * 16)
# Fix 2: moving access within the context
with open(file_name) as f:
    lines = (line.strip('\n') for line in f)

    for l in lines:
        print(l)

print("~^" * 16)


# Fix 3: Encapsulating it within a generator
def send_lines(file_name):
    with open(file_name) as f:
        for line in f:
            yield line.strip('\n')


for l in send_lines(file_name):
    print(l)

"Context Managers are great,
Context Managers are our Friend.
So why not use them
~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^
"Context Managers are great,
Context Managers are our Friend.
So why not use them
~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^~^
"Context Managers are great,
Context Managers are our Friend.
So why not use them


Now, lets learn about how to create our own context manager's. This will also helps us in understanding them in details. 

## Creating a Context Manager class

To create a context manager class, two functions are required, `__enter__` and `__exit__`, where initialization instructions are written in `__enter__` and cleanup instructions are written in `__exit__`.
Lets look at sample example below

In [79]:
import sqlite3 as sqlite

class Questy(object):
    def __init__(self, filename):
        self.filename = filename
    
    def __enter__(self):
        self.conn = sqlite.connect(self.filename)
        self.c = self.conn.cursor()
        return self.c
        
    def __exit__(self, exit_type, exit_value, exit_traceback):
        try:
            print("exit_type:", exit_type, "\nexit_value:", exit_value, "\nexit_traceback:", exit_traceback)
            print("Trying to close the cursor and connections.")
            self.c.close()
            self.conn.close()
        except Exception as e:
            print("Exception", e)

In [73]:
filename = "code/questions.sqlite"

with Questy(filename) as questy:
    topic_id = 1
    quests = questy.execute("SELECT * FROM questions WHERE topics_id = '%s' LIMIT 7" % topic_id)
    for q in quests:
        print(q[1])

What all types of exceptions thrown in internet driver?
Which type of `wait` remains active till the `webdriver` is not released
what are the unique navigation strategies used in webdriver
Which one is not the WebDriver’s anticipated situation? observe: WebDriver’s ExpectedConditions elegance is used to test expected conditions.
What are the WebDriver methods to manipulate browser based alerts?
what's are the similarities between WebDriver’s `close()` and `give up()` techniques?
which type of exception is dealt with for the duration of compile time?
None None None


We can see the above code is so easy now. Lets talk about the steps taken by the 

In [80]:
filename = "code/questions1.sqlite"

with Questy(filename) as questy:
    topic_id = 1
    quests = questy.execute("SELECT * FROM questions WHERE topics_id = '%s' LIMIT 7" % topic_id)
    for q in quests:
        print(q[1])

exit_type: <class 'sqlite3.OperationalError'> 
exit_value: no such table: questions 
exit_traceback: <traceback object at 0x7efce80c74c8>
Trying to close the cursor and connections.


OperationalError: no such table: questions

### Exceptions Handling

In the last example, we encounter an exception. In this section, we will try to learn ways to handle such exceptions.

As we have seen in the previous example, 

## Creating a Context Manager using contextlib

Python also allows us to convert `generators` to context managers using `contextlib` module 

## contextlib.closing(thing)

## contextlib.suppress(*exceptions)

## contextlib.redirect_stdout / redirect_stderr

## ExitStack

## Reentrant Context Managers

## Reference

- https://www.python.org/dev/peps/pep-0343
- https://docs.python.org/3/library/contextlib.html
- https://en.wikibooks.org/wiki/Python_Programming/Context_Managers
- http://preshing.com/20110920/the-python-with-statement-by-example
- https://stackoverflow.com/questions/6884991/how-to-delete-dir-created-by-python-tempfile-mkdtemp